# OS MCP Server Examples: Nottingham & Coventry

This notebook demonstrates the functionality of the **OS MCP Server** using real UK locations:
- **Nottingham NG1 7FG** - City center location 
- **Coventry CV1** - City center area

## What You'll Learn

- How to connect to the OS MCP Server
- Basic geospatial queries and searches
- Working with UK Ordnance Survey data
- Advanced filtering and location-based operations
- Real-world examples with actual UK postcodes

## Prerequisites

- OS API key from [OS Data Hub](https://osdatahub.os.uk/) set as `OS_API_KEY` environment variable
- Python environment with required dependencies (automatically installed)

## Automatic Setup

🚀 **This notebook will automatically:**
- Install required Python packages
- Start the OS MCP Server if not already running
- Handle all connections and setup
- Clean up when finished

**No manual server setup required!** Just run the cells in order and explore the data.

Let's explore the rich geospatial data available through the OS MCP Server! 🗺️

## 1. Install and Import Required Libraries

First, let's install any required packages and import the libraries we need for connecting to the OS MCP Server and handling geospatial data.

In [1]:
# Install required packages (if not already installed)
import subprocess
import sys

def install_package(package):
    try:
        __import__(package)
        print(f"✅ {package} already installed")
    except ImportError:
        print(f"📦 Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Install all required packages
required_packages = [
    "mcp",
    "matplotlib", 
    "pandas",
    "numpy"
]

print("🔄 Checking and installing required packages...")
for package in required_packages:
    install_package(package)

print("\n✅ All packages checked/installed!")

# Import required libraries
import asyncio
import logging
import json
import os
from typing import Dict, Any, List
import matplotlib.pyplot as plt
import pandas as pd

# Import MCP client libraries
from mcp.client.streamable_http import streamablehttp_client
from mcp import ClientSession

print("✅ All libraries imported successfully!")

# Configure logging for better visibility
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Enable matplotlib inline plotting for Jupyter
try:
    get_ipython().run_line_magic('matplotlib', 'inline')
    print("✅ Matplotlib inline plotting enabled")
except:
    print("ℹ️ Not in Jupyter environment - matplotlib will use default backend")

🔄 Checking and installing required packages...
✅ mcp already installed
📦 Installing matplotlib...
Defaulting to user installation because normal site-packages is not writeable
✅ mcp already installed
📦 Installing matplotlib...
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.9/107.9 kB 2.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.9/107.9 kB 2.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 20.4 MB/s eta 0:00:00
   ━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.3/8.5 MB 7.7 MB/s eta 0:00:02  Downloading pillow-11.3.0-cp311-cp311-manylinux_2_27_aarch64.manylinux_2_28_aarch64.whl.metadata (9.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 15.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.2/329.2 kB 43.7 MB/s eta 0:00:00
   


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 2.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 2.3 MB/s eta 0:00:00a 0:00:01
   ╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.2/11.8 MB 6.9 MB/s eta 0:00:02Collecting pytz>=2020.1 (from pandas)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 29.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 29.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 26.4 MB/s eta 0:00:00
✅ numpy already installed

✅ All packages checked/installed!
✅ numpy already installed

✅ All packages checked/installed!



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


✅ All libraries imported successfully!
✅ Matplotlib inline plotting enabled


## 2. Initialize OS MCP Client and Auto-Start Server

The next cells will:
1. **Configure connection settings** for the OS MCP Server
2. **Automatically check** if the server is running
3. **Start the server** if needed (no manual setup required!)
4. **Verify** everything is ready for data exploration

Make sure you have your OS API key set in the `OS_API_KEY` environment variable. The notebook handles everything else automatically! 🎯

In [2]:
# Configuration for OS MCP Server
SERVER_URL = "http://localhost:8000/mcp/"
HEADERS = {"Authorization": "Bearer dev-token"}

# Example locations we'll be working with
LOCATIONS = {
    "nottingham": {
        "name": "Nottingham NG1 7FG",
        "postcode": "NG1 7FG", 
        "lat": 52.9548,
        "lon": -1.1543,
        "bbox": [-1.16, 52.95, -1.15, 52.96]  # [min_lon, min_lat, max_lon, max_lat]
    },
    "coventry": {
        "name": "Coventry CV1",
        "postcode": "CV1",
        "lat": 52.4081,
        "lon": -1.5101,
        "bbox": [-1.52, 52.40, -1.50, 52.42]
    }
}

# Helper function to extract text from MCP responses
def extract_text_from_result(result):
    """Safely extract text from MCP tool result"""
    try:
        if result.content and len(result.content) > 0:
            content = result.content[0]
            if hasattr(content, 'text'):
                return content.text
            else:
                return str(content)
        return "No response"
    except Exception as e:
        return f"Error extracting text: {e}"

# Check environment variables
if not os.environ.get("OS_API_KEY"):
    print("⚠️ WARNING: OS_API_KEY environment variable not set!")
    print("💡 Get your API key from: https://osdatahub.os.uk/")
else:
    print("✅ OS_API_KEY environment variable detected")

print(f"🌍 Example locations configured:")
for key, loc in LOCATIONS.items():
    print(f"  📍 {loc['name']} at ({loc['lat']}, {loc['lon']})")
    
print(f"🔌 Server URL: {SERVER_URL}")
print(f"🔑 Authentication: Bearer token configured")

✅ OS_API_KEY environment variable detected
🌍 Example locations configured:
  📍 Nottingham NG1 7FG at (52.9548, -1.1543)
  📍 Coventry CV1 at (52.4081, -1.5101)
🔌 Server URL: http://localhost:8000/mcp/
🔑 Authentication: Bearer token configured


In [ ]:
# Check if OS MCP Server is running and start it if needed
import subprocess
import time
import requests
import signal
import os
from pathlib import Path

def is_server_running(url="http://localhost:8000", timeout=2):
    """Check if the OS MCP Server is running"""
    try:
        response = requests.get(f"{url}/", timeout=timeout)
        return True
    except:
        return False

def start_server():
    """Start the OS MCP Server in the background"""
    print("🚀 Starting OS MCP Server...")
    
    # Get the path to the server script
    current_dir = Path.cwd()
    if current_dir.name == "examples":
        server_path = current_dir.parent / "src" / "server.py"
    else:
        server_path = current_dir / "src" / "server.py"
    
    if not server_path.exists():
        print(f"❌ Server script not found at {server_path}")
        return None
    
    # Start the server process
    try:
        # Start server in background with streamable-http transport
        process = subprocess.Popen([
            "python", str(server_path),
            "--transport", "streamable-http",
            "--host", "0.0.0.0", 
            "--port", "8000"
        ], 
        stdout=subprocess.PIPE, 
        stderr=subprocess.PIPE,
        preexec_fn=os.setsid  # Create new process group
        )
        
        print(f"✅ Server process started with PID: {process.pid}")
        
        # Wait a bit for server to start
        print("⏱️ Waiting for server to start...")
        for i in range(10):
            time.sleep(1)
            if is_server_running():
                print("✅ Server is now running and responding!")
                return process
            print(f"   Waiting... ({i+1}/10)")
        
        print("⚠️ Server started but may not be fully ready yet")
        return process
        
    except Exception as e:
        print(f"❌ Failed to start server: {e}")
        return None

# Check if server is already running
print("🔍 Checking if OS MCP Server is already running...")
if is_server_running():
    print("✅ OS MCP Server is already running on port 8000")
    server_process = None
else:
    print("⚠️ OS MCP Server is not running - starting it now...")
    server_process = start_server()

# Store the process for cleanup later
if 'server_process' not in globals():
    server_process = None

print(f"\n📋 Server Status Summary:")
print(f"   🌐 Server URL: {SERVER_URL}")
print(f"   🔑 Authentication: Bearer token")
print(f"   📊 Process: {'Running' if server_process else 'Already running or failed'}")
print(f"   ✅ Ready for connections!")

# Final verification
if is_server_running():
    print("\n🎉 OS MCP Server is ready for notebook operations!")
else:
    print("\n❌ Server may not be responding - please check the terminal for errors")
    print("💡 You can also start the server manually with:")
    print("   cd /workspaces/os-mcp && python src/server.py --transport streamable-http --port 8000")

## 3. Connect to OS MCP Server and Basic Operations

Now we'll establish a connection to the server. The server should already be running from the previous cell, but if there are any connection issues, the notebook will provide helpful diagnostics.

In [3]:
async def connect_to_server():
    """Establish connection to OS MCP Server"""
    print("🔌 Connecting to OS MCP Server...")
    
    try:
        # Create client connection
        client_manager = streamablehttp_client(SERVER_URL, headers=HEADERS)
        read_stream, write_stream, get_session_id = await client_manager.__aenter__()
        
        # Create MCP session
        session = ClientSession(read_stream, write_stream)
        await session.__aenter__()
        
        # Initialize the session
        init_result = await session.initialize()
        session_id = get_session_id()
        
        print(f"✅ Connected successfully! Session ID: {session_id}")
        
        return session, client_manager
        
    except Exception as e:
        print(f"❌ Connection failed: {e}")
        raise

async def test_basic_operations(session):
    """Test basic server operations"""
    print("\n🧪 Testing basic operations...")
    
    # 1. Hello world test
    print("\n👋 Testing hello world...")
    try:
        result = await session.call_tool("hello_world", {"name": "Nottingham Explorer"})
        response = extract_text_from_result(result)
        print(f"✅ Hello world response: {response}")
    except Exception as e:
        print(f"❌ Hello world failed: {e}")
    
    # 2. Check API key
    print("\n🔑 Checking API key status...")
    try:
        result = await session.call_tool("check_api_key", {})
        response = extract_text_from_result(result)
        api_status = json.loads(response)
        if api_status.get("status") == "success":
            print(f"✅ {api_status.get('message')}")
        else:
            print(f"❌ {api_status.get('message')}")
    except Exception as e:
        print(f"❌ API key check failed: {e}")
    
    # 3. List available tools
    print("\n🔍 Listing available tools...")
    try:
        tools_result = await session.list_tools()
        tools = [tool.name for tool in tools_result.tools]
        print(f"✅ Found {len(tools)} tools:")
        for i, tool in enumerate(tools, 1):
            print(f"  {i}. {tool}")
    except Exception as e:
        print(f"❌ Tool listing failed: {e}")

# Run the connection and basic tests
session = None
client_manager = None

try:
    # This needs to be run in an async context
    session, client_manager = await connect_to_server()
    await test_basic_operations(session)
    print("\n🎉 Basic operations completed successfully!")
    
except Exception as e:
    print(f"💥 Basic operations failed: {e}")
    if session:
        try:
            await session.__aexit__(None, None, None)
        except:
            pass
    if client_manager:
        try:
            await client_manager.__aexit__(None, None, None) 
        except:
            pass

🔌 Connecting to OS MCP Server...


CancelledError: Cancelled by cancel scope ffff6c137390

## 4. Get Workflow Context and Available Collections

Before we can perform any searches, we need to get the workflow context. This is a required step that tells us what data collections are available and how to filter them effectively.

In [ ]:
async def get_workflow_context(session):
    """Get the workflow context - required before any searches"""
    print("🎯 Getting workflow context (required for all searches)...")
    
    try:
        result = await session.call_tool("get_workflow_context", {})
        response_text = extract_text_from_result(result)
        context_data = json.loads(response_text)
        
        print("✅ Workflow context retrieved successfully!")
        
        # Show available collections
        if "available_collections" in context_data:
            collections = context_data["available_collections"]
            print(f"\n📊 Found {len(collections)} data collections:")
            
            # Create a summary dataframe
            collection_summary = []
            for coll_id, coll_info in collections.items():
                collection_summary.append({
                    "Collection ID": coll_id,
                    "Title": coll_info.get("title", "No title"),
                    "Has Enum Filters": coll_info.get("has_enum_filters", False),
                    "Total Queryables": coll_info.get("total_queryables", 0),
                    "Enum Count": coll_info.get("enum_count", 0)
                })
            
            df = pd.DataFrame(collection_summary)
            print(df.to_string(index=False))
            
        return context_data
        
    except Exception as e:
        print(f"❌ Failed to get workflow context: {e}")
        raise

async def list_collections_detail(session):
    """Get detailed collection information"""
    print("\n📚 Getting detailed collection information...")
    
    try:
        result = await session.call_tool("list_collections", {})
        response_text = extract_text_from_result(result)
        collections_data = json.loads(response_text)
        
        if "collections" in collections_data:
            collections = collections_data["collections"]
            print(f"✅ Found {len(collections)} collections with details:")
            
            for i, collection in enumerate(collections[:5], 1):  # Show first 5
                coll_id = collection.get("id", "Unknown ID")
                title = collection.get("title", "No title")
                print(f"  {i}. {coll_id}: {title}")
            
            if len(collections) > 5:
                print(f"  ... and {len(collections) - 5} more collections")
                
        return collections_data
        
    except Exception as e:
        print(f"❌ Failed to list collections: {e}")
        return {}

# Get workflow context and collections
if session:
    try:
        workflow_context = await get_workflow_context(session)
        collections_data = await list_collections_detail(session)
        print("\n🎉 Workflow context and collections loaded successfully!")
        
        # Show some key information about filtering
        if "QUICK_FILTERING_GUIDE" in workflow_context:
            print("\n💡 Quick filtering tips:")
            print("  • Use exact enum values for precise filtering")
            print("  • Always explain your plan before making searches")
            print("  • Use the 'filter' parameter for all filtering operations")
            
    except Exception as e:
        print(f"💥 Workflow context failed: {e}")
else:
    print("❌ No active session - please run the connection cell first")

## 5. Location-Based Searches: Nottingham NG1 7FG and Coventry CV1

Now let's perform real searches around our example locations. We'll search for streets, buildings, and land use features in both Nottingham and Coventry.

In [ ]:
async def search_streets_around_location(session, location_name, bbox):
    """Search for streets around a specific location"""
    print(f"🛣️ Searching for streets around {location_name}...")
    
    try:
        bbox_str = ",".join(map(str, bbox))
        
        result = await session.call_tool("search_features", {
            "collection_id": "trn-ntwk-street-1",
            "bbox": bbox_str,
            "limit": 10
        })
        
        response_text = extract_text_from_result(result)
        street_data = json.loads(response_text)
        
        if "features" in street_data:
            features = street_data["features"]
            print(f"✅ Found {len(features)} streets around {location_name}")
            
            # Create a summary of streets found
            street_summary = []
            for feature in features[:5]:  # Show first 5
                props = feature.get("properties", {})
                street_summary.append({
                    "Street Name": props.get("designatedname1_text", "Unnamed street"),
                    "Road Classification": props.get("roadclassification", "Unknown"),
                    "Operational State": props.get("operationalstate", "Unknown"),
                    "Road Number": props.get("roadnumber", "N/A")
                })
            
            if street_summary:
                df = pd.DataFrame(street_summary)
                print(df.to_string(index=False))
            
            return street_data
        else:
            print(f"⚠️ No streets found around {location_name}")
            return {}
            
    except Exception as e:
        print(f"❌ Street search failed for {location_name}: {e}")
        return {}

async def find_land_use_around_location(session, location_name, bbox, land_use_type):
    """Find specific land use features around a location"""
    print(f"\n🏢 Searching for {land_use_type} around {location_name}...")
    
    try:
        bbox_str = ",".join(map(str, bbox))
        
        result = await session.call_tool("search_features", {
            "collection_id": "lus-fts-site-1",
            "bbox": bbox_str,
            "filter": f"oslandusetertiarygroup = '{land_use_type}'",
            "limit": 5
        })
        
        response_text = extract_text_from_result(result)
        land_use_data = json.loads(response_text)
        
        if "features" in land_use_data:
            features = land_use_data["features"]
            if features:
                print(f"✅ Found {len(features)} {land_use_type.lower()} locations around {location_name}")
                
                for i, feature in enumerate(features, 1):
                    props = feature.get("properties", {})
                    site_name = props.get("distname1", f"{land_use_type} site {i}")
                    print(f"  {i}. {site_name}")
            else:
                print(f"ℹ️ No {land_use_type.lower()} locations found around {location_name}")
        
        return land_use_data
        
    except Exception as e:
        print(f"❌ {land_use_type} search failed for {location_name}: {e}")
        return {}

# Perform searches for both locations
if session:
    print("🌍 Performing location-based searches...")
    
    for location_key, location_data in LOCATIONS.items():
        print(f"\n{'='*60}")
        print(f"📍 Exploring {location_data['name']}")
        print(f"📊 Coordinates: {location_data['lat']}, {location_data['lon']}")
        print(f"📦 Search area: {location_data['bbox']}")
        print(f"{'='*60}")
        
        try:
            # Search for streets
            streets = await search_streets_around_location(
                session, 
                location_data['name'], 
                location_data['bbox']
            )
            
            # Search for retail locations
            retail = await find_land_use_around_location(
                session,
                location_data['name'],
                location_data['bbox'],
                "Retail"
            )
            
            # Search for transport hubs
            transport = await find_land_use_around_location(
                session,
                location_data['name'], 
                location_data['bbox'],
                "Transport"
            )
            
        except Exception as e:
            print(f"❌ Search failed for {location_data['name']}: {e}")
    
    print("\n🎉 Location-based searches completed!")
else:
    print("❌ No active session - please run the connection cell first")

## 6. Advanced Filtering and Query Examples

Let's explore more sophisticated filtering capabilities using CQL (Common Query Language) expressions to find specific types of features.

In [ ]:
async def demonstrate_advanced_filters(session):
    """Demonstrate advanced CQL filtering capabilities"""
    print("🔍 Demonstrating advanced filtering capabilities...")
    
    filter_examples = [
        {
            "name": "A Roads in Nottingham",
            "collection": "trn-ntwk-street-1",
            "filter": "roadclassification = 'A Road'",
            "bbox": LOCATIONS["nottingham"]["bbox"],
            "description": "Find A roads specifically in Nottingham area"
        },
        {
            "name": "Open A Roads",
            "collection": "trn-ntwk-street-1", 
            "filter": "roadclassification = 'A Road' AND operationalstate = 'Open'",
            "description": "Find A roads that are currently operational"
        },
        {
            "name": "High Streets",
            "collection": "trn-ntwk-street-1",
            "filter": "designatedname1_text LIKE '%High%'",
            "description": "Find streets with 'High' in the name"
        },
        {
            "name": "Retail Areas in Coventry",
            "collection": "lus-fts-site-1",
            "filter": "oslandusetiera = 'Retail'",
            "bbox": LOCATIONS["coventry"]["bbox"],
            "description": "Find retail areas specifically in Coventry"
        }
    ]
    
    results_summary = []
    
    for example in filter_examples:
        print(f"\n📋 {example['name']}")
        print(f"   {example['description']}")
        print(f"   Filter: {example['filter']}")
        
        try:
            params = {
                "collection_id": example['collection'],
                "filter": example['filter'],
                "limit": 5
            }
            
            if 'bbox' in example:
                params['bbox'] = ",".join(map(str, example['bbox']))
                print(f"   Area: {example['bbox']}")
            
            result = await session.call_tool("search_features", params)
            response_text = extract_text_from_result(result)
            data = json.loads(response_text)
            
            feature_count = len(data.get("features", []))
            results_summary.append({
                "Query": example['name'],
                "Collection": example['collection'],
                "Results Found": feature_count,
                "Status": "✅ Success" if feature_count > 0 else "ℹ️ No results"
            })
            
            if feature_count > 0:
                print(f"   ✅ Found {feature_count} results")
                
                # Show details for first few results
                features = data["features"][:2]
                for i, feature in enumerate(features, 1):
                    props = feature.get("properties", {})
                    
                    if example['collection'] == "trn-ntwk-street-1":
                        name = props.get("designatedname1_text", "Unnamed")
                        classification = props.get("roadclassification", "Unknown")
                        state = props.get("operationalstate", "Unknown")
                        print(f"      {i}. {name} ({classification}, {state})")
                    
                    elif example['collection'] == "lus-fts-site-1":
                        name = props.get("distname1", "Unnamed location")
                        land_use = props.get("oslandusetiera", "Unknown type")
                        print(f"      {i}. {name} ({land_use})")
            else:
                print(f"   ℹ️ No results found")
                
        except Exception as e:
            print(f"   ❌ Error: {e}")
            results_summary.append({
                "Query": example['name'],
                "Collection": example['collection'],
                "Results Found": 0,
                "Status": f"❌ Error: {str(e)[:50]}..."
            })
    
    # Show summary table
    print(f"\n📊 Summary of Advanced Filter Results:")
    if results_summary:
        df = pd.DataFrame(results_summary)
        print(df.to_string(index=False))
    
    return results_summary

# Run advanced filtering examples
if session:
    try:
        filter_results = await demonstrate_advanced_filters(session)
        print("\n🎉 Advanced filtering examples completed!")
    except Exception as e:
        print(f"💥 Advanced filtering failed: {e}")
else:
    print("❌ No active session - please run the connection cell first")

## 7. Postcode and Address Searches

Let's search for address data using our specific postcodes: NG1 7FG (Nottingham) and CV1 (Coventry area).

In [ ]:
async def search_addresses_by_postcode(session, postcode_pattern, location_name):
    """Search for addresses by postcode pattern"""
    print(f"🏠 Searching for addresses in {postcode_pattern} ({location_name})...")
    
    try:
        result = await session.call_tool("search_features", {
            "collection_id": "adr-fts-addressbasepremium-1",
            "filter": f"postcode LIKE '{postcode_pattern}%'",
            "limit": 10
        })
        
        response_text = extract_text_from_result(result)
        address_data = json.loads(response_text)
        
        if "features" in address_data:
            features = address_data["features"]
            if features:
                print(f"✅ Found {len(features)} addresses in {postcode_pattern}")
                
                # Create address summary
                address_summary = []
                for i, feature in enumerate(features[:5], 1):  # Show first 5
                    props = feature.get("properties", {})
                    address_summary.append({
                        "Address": props.get("address", "No address"),
                        "Postcode": props.get("postcode", "No postcode"),
                        "UPRN": props.get("uprn", "No UPRN"),
                        "Classification": props.get("classification", "Unknown")
                    })
                
                if address_summary:
                    df = pd.DataFrame(address_summary)
                    print(df.to_string(index=False))
                
                return features
            else:
                print(f"ℹ️ No addresses found for {postcode_pattern}")
        
        return []
        
    except Exception as e:
        print(f"❌ Address search failed for {postcode_pattern}: {e}")
        return []

async def get_feature_by_id(session, collection_id, feature_id):
    """Get specific feature details by ID"""
    print(f"🔍 Getting feature details for {feature_id} in {collection_id}...")
    
    try:
        result = await session.call_tool("get_feature", {
            "collection_id": collection_id,
            "feature_id": feature_id
        })
        
        response_text = extract_text_from_result(result)
        feature_data = json.loads(response_text)
        
        if "properties" in feature_data:
            print("✅ Feature details retrieved:")
            props = feature_data["properties"]
            
            # Show key properties
            for key, value in list(props.items())[:5]:  # Show first 5 properties
                print(f"   {key}: {value}")
            
            if len(props) > 5:
                print(f"   ... and {len(props) - 5} more properties")
                
        return feature_data
        
    except Exception as e:
        print(f"❌ Feature retrieval failed: {e}")
        return {}

# Search for addresses in our example postcodes
if session:
    print("🏠 Performing postcode and address searches...")
    
    # Search Nottingham NG1 7FG area
    nottingham_addresses = await search_addresses_by_postcode(
        session, "NG1", "Nottingham"
    )
    
    print("\n" + "="*50)
    
    # Search Coventry CV1 area
    coventry_addresses = await search_addresses_by_postcode(
        session, "CV1", "Coventry" 
    )
    
    # If we found any addresses, get detailed info for the first one
    if nottingham_addresses:
        print(f"\n🔍 Getting detailed information for first Nottingham address...")
        first_address = nottingham_addresses[0]
        address_props = first_address.get("properties", {})
        if "id" in first_address:
            feature_details = await get_feature_by_id(
                session,
                "adr-fts-addressbasepremium-1", 
                first_address["id"]
            )
    
    print("\n🎉 Postcode and address searches completed!")
    
else:
    print("❌ No active session - please run the connection cell first")

## 8. Visualization and Summary

Let's create some simple visualizations of our search results and summarize what we've discovered about Nottingham NG1 7FG and Coventry CV1.

In [ ]:
# Create a summary visualization of our locations
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Plot 1: Location coordinates
locations_df = pd.DataFrame([
    {"Location": "Nottingham NG1 7FG", "Latitude": 52.9548, "Longitude": -1.1543},
    {"Location": "Coventry CV1", "Latitude": 52.4081, "Longitude": -1.5101}
])

ax1.scatter(locations_df["Longitude"], locations_df["Latitude"], s=100, alpha=0.7)
for i, row in locations_df.iterrows():
    ax1.annotate(row["Location"], (row["Longitude"], row["Latitude"]), 
                xytext=(5, 5), textcoords='offset points', fontsize=9)

ax1.set_xlabel("Longitude")
ax1.set_ylabel("Latitude") 
ax1.set_title("Example Locations")
ax1.grid(True, alpha=0.3)

# Plot 2: Bounding box visualization
bbox_data = []
for name, loc in LOCATIONS.items():
    bbox = loc["bbox"]
    bbox_data.append({
        "Location": loc["name"],
        "Bbox Width": bbox[2] - bbox[0],  # max_lon - min_lon
        "Bbox Height": bbox[3] - bbox[1]  # max_lat - min_lat
    })

bbox_df = pd.DataFrame(bbox_data)
bbox_df.plot(x="Location", y=["Bbox Width", "Bbox Height"], kind="bar", ax=ax2)
ax2.set_title("Search Area Sizes")
ax2.set_ylabel("Degrees")
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Summary statistics
print("📊 OS MCP Server Examples Summary")
print("="*50)
print(f"📍 Locations explored: {len(LOCATIONS)}")

for name, loc in LOCATIONS.items():
    print(f"\n🌍 {loc['name']}:")
    print(f"   📍 Coordinates: {loc['lat']}, {loc['lon']}")
    print(f"   📦 Search area: {loc['bbox']}")
    print(f"   📮 Postcode: {loc['postcode']}")

print(f"\n🔧 Operations performed:")
print(f"   ✅ Server connection and authentication")
print(f"   ✅ Workflow context retrieval")
print(f"   ✅ Collection discovery")
print(f"   ✅ Location-based street searches")
print(f"   ✅ Land use feature searches")
print(f"   ✅ Advanced CQL filtering")
print(f"   ✅ Postcode-based address searches")

print(f"\n💡 Key learnings:")
print(f"   • Always call get_workflow_context() first")
print(f"   • Use exact enum values for precise filtering")
print(f"   • Bounding boxes help focus searches geographically")
print(f"   • CQL filters enable sophisticated queries")
print(f"   • Multiple data collections available (streets, addresses, land use)")

print(f"\n🎯 Next steps:")
print(f"   • Explore additional collections and their queryables")
print(f"   • Try more complex CQL filter expressions")
print(f"   • Experiment with different geographic areas")
print(f"   • Integrate with mapping libraries for visualization")

print(f"\n🎉 OS MCP Server examples completed successfully!")
print(f"   Ready to explore UK geospatial data! 🗺️")

## 9. Cleanup and Next Steps

Finally, let's clean up our connections and provide guidance for further exploration.

In [ ]:
# Clean up connections
if session:
    try:
        await session.__aexit__(None, None, None)
        print("✅ MCP session closed")
    except Exception as e:
        print(f"⚠️ Warning: Error closing session: {e}")

if client_manager:
    try:
        await client_manager.__aexit__(None, None, None)
        print("✅ Client connection closed")
    except Exception as e:
        print(f"⚠️ Warning: Error closing client: {e}")

# Stop the server if we started it in this notebook
if 'server_process' in globals() and server_process is not None:
    print("\n🛑 Stopping OS MCP Server that was started by this notebook...")
    try:
        import os
        import signal
        # Kill the entire process group to ensure all child processes are terminated
        os.killpg(os.getpgid(server_process.pid), signal.SIGTERM)
        server_process.wait(timeout=5)  # Wait up to 5 seconds for graceful shutdown
        print("✅ Server stopped successfully")
    except subprocess.TimeoutExpired:
        print("⚠️ Server didn't stop gracefully, forcing termination...")
        os.killpg(os.getpgid(server_process.pid), signal.SIGKILL)
        print("✅ Server force-stopped")
    except Exception as e:
        print(f"⚠️ Warning: Error stopping server: {e}")
        print("💡 You may need to stop it manually if it's still running")
    
    server_process = None
else:
    print("\n⚠️ Server was not started by this notebook - leaving it running")
    print("💡 If you want to stop the server, use Ctrl+C in the terminal where it's running")

print("\n🎯 Exploration Complete!")
print("\n📚 Additional Resources:")
print("   • OS Data Hub: https://osdatahub.os.uk/")
print("   • OS MCP Server GitHub: Your project repository")
print("   • More examples: Check the examples/ directory")

print("\n🛠️ Try These Next:")
print("   • Explore different collections with get_collection_info()")
print("   • Try get_collection_queryables() to see all available filters")
print("   • Experiment with different bounding boxes")
print("   • Combine multiple filter criteria with AND/OR")
print("   • Search for specific street names or building types")

print("\n💡 Pro Tips:")
print("   • Use tighter bounding boxes for faster searches")
print("   • Check enum values in workflow context for exact filtering")
print("   • The server has built-in rate limiting for protection") 
print("   • Always handle errors gracefully in production code")

print("\n🌟 Happy exploring UK geospatial data with OS MCP Server! 🗺️")